In [ ]:
import os
from IPython.display import clear_output


notebook_dir = "/home/balabaevvl/courses/project/FITS/src/"
os.chdir(notebook_dir)

GPUs = [
    "GPU-e83bd31b-fcb9-b8de-f617-2d717619413b",
    "GPU-5a9b7750-9f85-49a5-3aae-fe07b1b7661d",
    "GPU-fe2d8dfd-06f2-a5c4-a7fd-4a5f23947005",
    "GPU-0c320096-21ee-4060-8731-826ca2febfab",
    "GPU-baef952c-6609-aace-3b78-e4e07788d5de",
    "GPU-3979d65b-c238-4e9c-0c1c-1aa3f05c56a1",
    "GPU-6c76a2c5-5375-aa06-11d4-0fddfac30e91",
]
os.environ["CUDA_VISIBLE_DEVICES"] = f"{GPUs[2]}"

In [ ]:
from fits.dataframes.download import DownloadDatasetAirQuality

DownloadDatasetAirQuality()

In [ ]:
# import pandas as pd
# from fits.config import DatasetsPaths

# df = pd.read_csv(
#     DatasetsPaths.pm25.value,
#     index_col="datetime",
#     parse_dates=True,
# )

# df.head()

In [ ]:
# from fits.dataframes.dataset import ModelMode, DatasetAirQuality

# dataset = DatasetAirQuality(ModelMode.train)

# for sample in dataset:
#     break
# sample

In [ ]:
import torch
from fits.modelling.framework import Train, Evaluate
from fits.dataframes.dataset import DatasetAirQuality
from fits.dataframes.dataloader import ForecastingDataLoader
from fits.modelling.comparison import (
    CalculateParams,
    ReadMetrics,
    VisualizeForecastSample,
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NSAMPLE = 5

In [ ]:
train_loader, valid_loader, test_loader = ForecastingDataLoader(
    DatasetAirQuality, batch_size=128
)  # shape=[B, L=48, K=36]
normalization_stats = train_loader.dataset.normalization_stats

# VAR

In [ ]:
from fits.modelling.VAR.model import SeasonalVAR, VARConfig

var = SeasonalVAR(VARConfig(lag_order=12)).to(device)

print(f"Total params: {CalculateParams(var)}")

In [ ]:
Train(var, train_loader, valid_loader, epochs=60)

In [ ]:
Evaluate(var, test_loader, normalization_stats, nsample=NSAMPLE)

In [ ]:
eval_foldername = "SeasonalVAR_20260105_145357"

ReadMetrics(eval_foldername, nsample=NSAMPLE)
# VisualizeForecastSample(eval_foldername, nsample=NSAMPLE, sample_index=10)

# CSDI

In [ ]:
from fits.modelling.CSDI.adapter import CSDIAdapter


csdi = CSDIAdapter().to(device)
print(f"Total params: {CalculateParams(csdi)}")

In [ ]:
Train(csdi, train_loader, valid_loader, epochs=300)

In [ ]:
# model_foldername = "CSDIAdapter_20260108_093514"

# state = torch.load(
#     f"../data/models/training/{model_foldername}/model.pth",
#     map_location=device,
# )

# csdi.load_state_dict(state)
# csdi.to(device)

# clear_output()

In [ ]:
Evaluate(csdi, test_loader, normalization_stats, nsample=NSAMPLE)

In [ ]:
eval_foldername = "CSDIAdapter_20260108_140029"

ReadMetrics(eval_foldername, nsample=NSAMPLE)
# VisualizeForecastSample(eval_foldername, nsample=NSAMPLE, sample_index=10)

In [ ]:
import gc, torch

del csdi
gc.collect()
torch.cuda.empty_cache()

# DiffusionTS

In [ ]:
from fits.modelling.DiffusionTS.adapter import DiffusionTSAdapter, DiffusionTSConfig


diffusionts = DiffusionTSAdapter().to(device)
print(f"Total params: {CalculateParams(diffusionts)}")

In [ ]:
Train(diffusionts, train_loader, valid_loader, epochs=1500)

In [ ]:
# model_foldername = "DiffusionTSAdapter_20260104_211002"

# state = torch.load(
#     f"../data/models/training/{model_foldername}/model.pth",

#     map_location=device,
# )

# diffusionts.load_state_dict(state)
# diffusionts.to(device)

# clear_output()

In [ ]:
Evaluate(diffusionts, test_loader, normalization_stats, nsample=NSAMPLE)
# TODO: epochs 100->300; enc=4 dec=3; timesteps=1000
# TODO: try lr=1e-4 -> works the same
# TODO: langevin_coef=0.0

In [ ]:
# eval_foldername = "DiffusionTSAdapter_20260105_173500"
# eval_foldername = "DiffusionTSAdapter_500_500"
eval_foldername = "DiffusionTSAdapter_500_500_upd_langevein"
# eval_foldername = "DiffusionTSAdapter_500_500_no_langevein"

ReadMetrics(eval_foldername, nsample=NSAMPLE)
# VisualizeForecastSample(eval_foldername, nsample=NSAMPLE, sample_index=10)

# FM-TS

In [ ]:
from fits.modelling.FMTS.adapter import FMTSAdapter, FMTSConfig


fmts = FMTSAdapter(FMTSConfig(n_layer_enc=1, n_layer_dec=1, d_model=4 * 8)).to(device)
print(f"Total params: {CalculateParams(fmts)}")

In [ ]:
Train(fmts, train_loader, valid_loader, epochs=600, weight_decay=0)

In [ ]:
model_foldername = "FMTSAdapter_20260108_005432"

state = torch.load(
    f"../data/models/training/{model_foldername}/best_model.pth",
    map_location=device,
)

fmts.load_state_dict(state)
fmts.to(device)

clear_output()

In [ ]:
Evaluate(fmts, test_loader, normalization_stats, nsample=NSAMPLE)
# RMSE: 62.116167960730074
# MAE: 41.6467024492886
# CRPS: 0.42974482084575455
# CRPS_sum: 0.37690636986180354


# RMSE: 51.043970189615465
# MAE: 37.49423109614115
# CRPS: 0.40085478832847193
# CRPS_sum: 0.3570429902327688

In [ ]:
eval_foldername = "..."

ReadMetrics(eval_foldername, nsample=NSAMPLE)
# VisualizeForecastSample(eval_foldername, nsample=NSAMPLE, sample_index=10)

In [ ]:
# TODO: Comparison of truth VS forecasted in:
# PSA plot
# t-SNE plot
# Data Density